# Dependencies

In [ ]:
!pip install tensorflow==2.13 gym keras-rl2 gym[atari]

# Test the environment

In [ ]:
import gym
import random

In [ ]:
env = gym.make('SpaceInvaders-v0')
height, width, channels = env.observation_space.shape # extracting the shape of the environment
actions = env.action_space.n

In [ ]:
env.unwrapped.get_action_meanings() # getting the actions that can be taken

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0,1,2,3,4,5])
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

# Create Deep learning model with Keras

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras.optimizers import Adam

In [ ]:
def build_model(heigh, width, channels, actions):
    model = Sequential()
    model.add(Conv2D(32, (8,8), strides=(4,4), activation='relu', input_shape=(3, height, width, channels)))
    model.add(Conv2D(64, (4,4), strides=(2,2), activation='relu'))
    model.add(Conv2D(64, (3,3), activation='relu'))
    model.add(Flatten())
    model.add.Dense(512, activation='relu')
    model.add(Dense(256, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
model = build_model(height, width, channels, actions)

# Build agent with Keras

In [ ]:
from rl.agents import DQNAgent
from rl.policy import LinearAnnealingPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
    policy = LinearAnnealingPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.2, nb_steps=10000) # policy that will be used to select actions is Epsilon Greedy which will select random actions with a probability of 0.1
    memory = SequentialMemory(limit=1000, window_length=3) # SequentialMemory will be used to store the experience of the agent

    # dqn is the agent that will be used to interact with the environment
    dqn = DQNAgent(model=model, memory=memory, policy=policy, enable_dueling_network=True, dueling_type='avg', nb_actions=actions, nb_steps_warmup=1000)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=0.0001))
dqn.fit(env, nb_steps=10000, visualize=False, verbose=2)

In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

# Reload model

In [ ]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)
del model, dqn
dqn.load_weights('dqn_weights.h5f')